In [2]:
# import cv2
# import numpy as np 
# import tensorflow as tf 
# from absl import logging
# from itertools import repeat
# from PIL import Image
# from tensorflow.keras import Model
# from tensorflow.keras.layers import Add, Concatenate, Lambda
# from tensorflow.keras.layers import Conv2D, Input, LeakyReLU
# from tensorflow.keras.layers import MaxPool2D, UpSampling2D, ZeroPadding2D
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.losses import binary_crossentropy
# from tensorflow.keras.losses import sparse_categorical_crossentropy
# yolo_iou_threshold = 0.6 
# yolo_score_threshold = 0.6 # Score threshold.
# weightyolov3 = '../weights/yolov3.weights' # Path to the file with weights.
# size = 416 # Image size.
# checkpoints = '../checkpoints/yolov3.tf' # Path to the checkpoint file.
# num_classes = 80 # The number of classes in the model.
# YOLO_V3_LAYERS = [
#     'yolo_darknet',
#     'yolo_conv_0',
#     'yolo_output_0',
#     'yolo_conv_1',
#     'yolo_output_1',
#     'yolo_conv_2',
#     'yolo_output_2'
# ]

In [3]:
# # Function to load the weights of the trained model.
# def load_darknet_weights(model, weights_file):
#     wf = open(weights_file, 'rb')
#     layers = YOLO_V3_LAYERS
#     for lay_name in layers:
#         s_model = model.get_layer(lay_name)
#     for i, layer in enumerate(s_model.layers):
#         if not layer.name.startswith('conv2d'):
#             continue
#             batch_n = None
#         if i + 1 < len(s_model.layers) and s_model.layers[i + 1].name.startswith('batch_n'):
#             batch_n = s_model.layers[i + 1]
#             logging.info("{}/{} {}".format(s_model.name, layer.name, 'bn' if batch_n else 'bias'))
#             ft = layer.filters
#             size = layer.kernel_size[0]
#             in_dim = layer.input_shape[-1]
#         if batch_n is None:
#                 conv_bias = np.fromfile(wf, dtype=np.float32, count=ft)
#         else:
#             bn_weight = np.fromfile(wf, dtype=np.float32, count=4*ft)
#             bn_weight = bn_weight.reshape((4, ft))[[1, 0, 2, 3]]
#             c_shape = (ft, in_dim, size, size)
#             c_weights = np.fromfile(wf, dtype=np.float32, count=np.product(c_shape))
#             c_weights = c_weights.reshape(c_shape).transpose([2, 3, 1, 0])
#         if batch_n is None:
#                 layer.set_weights([c_weights, c_bias])
#         else:
#                 layer.set_weights([c_weights])
#                 batch_n.set_weights(bn_weight)
#         assert len(wf.read()) == 0, 'declined!!'
#         wf.close()

In [4]:
# # Function for calculating IoU.
# def interval_overlap(int_1, int_2):
#     y1, y2 = int_1
#     y3, y4 = int_2
#     if y3 < y1:
#         return 0 if y4 < y1 else (min(y2,y4) - y1)
#     else:
#         return 0 if y2 < y3 else (min(y2,y4) - y3)
# def intersectionOverUnion(b1, b2):
#     inter_w = int_overlap([b1.xmin, b1.xmax], [b2.xmin, b2.xmax])
#     inter_h = int_overlap([b1.ymin, b1.ymax], [b2.ymin, b2.ymax])
#     inter_area = inter_w * inter_h
#     w1, h1 = b1.xmax-b1.xmin, b1.ymax-b1.ymin
#     w2, h2 = b2.xmax-b2.xmin, b2.ymax-b2.ymin
#     union_area = w1*h1 + w2*h2 - inter_area
#     return float(inter_area) / union_area
# class BatchNormalization(tf.keras.layers.BatchNormalization):
#     def call(self, x, train=False):
#         if train is None: train = tf.constant(False)
#         train = tf.logical_and(train, self.trainable)
#         return super().call(x, train)
#     # Define 3 anchor boxes for each cell.   
# y_anch = np.array([(10, 13), (16, 30), (33, 23), (30, 61), (62, 45),(57, 117), (114, 86), (154, 194), (373, 323)], np.float32) / 416
# y_anchor_mask = np.array([[6, 7, 8], [3, 4, 5], [0, 1, 2]])

In [5]:
# # Function for drawing bounding boxes.
# def draw_outputs(img, op, c_name, white_list=None):
#     box, score, label, nums = op
#     box, score, label, nums = box[0], score[0], label[0], nums[0]
#     wh = np.flip(img.shape[0:2])
#     for i in range(nums):
#         if c_name[int(classes[i])] not in white_list:
#             continue
#             x_1y_1 = tuple((np.array(box[i][0:2]) * wh).astype(np.int32))
#             x_2y_2 = tuple((np.array(box[i][2:4]) * wh).astype(np.int32))
#             ig = cv2.rectangle(ig, x_1y_1, x_2y_2, (255, 0, 0), 2)
#             ig = cv2.putText(img, '{} {:.4f}'.format(
#                 c_name[int(classes[i])], score[i]),
#                 x_1y_1, cv2.FONT_HERS_COMP_SMALL, 1, (0, 0, 255), 2)
#     return img

In [6]:
# def DarkConv(x, ft, size, stride=1, batch_n=True):
#     if stride == 1:
#             padding = 'same'
#     else:
#             x = ZeroPadding2D(((1, 0), (1, 0)))(x)
#             padding = 'valid'
#             x = Conv2D(ft,size,stride,padding,not batch_n,l2(0.0005))(x)
#     if batch_n:
#             x = BatchNormalization()(x)
#             x = LeakyReLU(alpha=0.1)(x)
#     return x

# def DarknetResidual(x, ft):
#     prev = x
#     x = DarkConv(x, ft // 2, 1)
#     x = DarkConv(x, ft, 3)
#     x = Add()([prev , x])
#     return x

# def DarkBlock(x, ft, block):
#     x = DarkConv(x, ft, 3, stride=2)
#     for _ in repeat(None, block):
#         x = DarknetResidual(x, ft)
#     return x

# def Darknet(name=None):
#     y = inputs = Input([None, None, 3])
#     y = DarkConv(y, 32, 3)
#     y = DarkBlock(y, 64, 1)
#     y = DarkBlock(y, 128, 2)
#     y = x_36 = DarknetBlock(y, 256, 8)
#     y = x_61 = DarknetBlock(y, 512, 8)
#     y = DarkBlock(y, 1024, 4)
#     return tf.keras.Model(inputs, (x_36, x_61, x), name=name)

# def YoloConv(ft, name=None):
#     def yolo_conv(x_in):
#         if isinstance(x_in, tuple):
#             inp = Input(x_in[0].shape[1:]), Input(x_in[1].shape[1:])
#             x, x_skip = inp
#             x = DarkConv(x, filters, 1)
#             x = UpSampling2D(2)(x)
#             x = Concatenate()([x, x_skip])
#         else:
#             x = Input(x_in.shape[1:])
#             x = DarkConv(x, filters, 1)
#             x = DarkConv(x, filters * 2, 3)
#             x = DarkConv(x, filters, 1)
#             x = DarkConv(x, filters * 2, 3)
#             x = DarkConv(x, filters, 1)
#         return Model(inputs, x, name=name)(x_in)
#     return yolo_conv

# def YoloOutput(ft, anch, label, name=None):
#     def yolo_op(x_in):
#         x = inp = Input(x_in.shape[1:])
#         x = DarkConv(x, ft * 2, 3)
#         x = DarkConv(x, anch * (label + 5), 1, batch_n=False)
#         return tf.keras.Model(inputs, x, name=name)(x_in)
#     return yolo_op

# def yolo_boxes(pred, anch, label):
#     g_size = tf.shape(pred)[1]
#     b_xy, b_wh, score, c_prob = tf.split(pred, (2, 2, 1, label), axis=-1)
#     b_xy = tf.sigmoid(b_xy)
#     sc = tf.sigmoid(sc)
#     c_prob = tf.sigmoid(c_prob)
#     pred_box = tf.concat((b_xy, b_wh), axis=-1)
#     gr = tf.meshgrid(tf.range(g_size), tf.range(g_size))
#     gr = tf.expand_dims(tf.stack(gr, axis=-1), axis=2)
#     b_xy = (b_xy + tf.cast(gr, tf.float32)) /  tf.cast(g_size, tf.float32)
#     b_wh = tf.exp(box_wh) * anch
#     b_x1y1 = b_xy - b_wh / 2
#     b_x2y2 = b_xy + b_wh / 2
#     bbox = tf.concat([b_x1y1, b_x2y2], axis=-1)
#     return bbox, score, c_prob, pred_box

In [7]:
# def nonMaximumSuppression(op, anch, mask, label):
#     boxes, conf, o_type = [], [], []
#     for output in op:
#         boxes.append(tf.reshape(op[0], (tf.shape(op[0])[0], -1, tf.shape(op[0])[-1])))
#         conf.append(tf.reshape(output[1], (tf.shape(op[1])[0], -1, tf.shape(op[1])[-1])))
#         o_type.append(tf.reshape(op[2], (tf.shape(op[2])[0], -1, tf.shape(op[2])[-1])))
#     bbox = tf.concat(boxes, axis=1)
#     confidence = tf.concat(conf, axis=1)
#     c_prob = tf.concat(o_type, axis=1)
#     scores = confidence * c_prob
#     boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
#         boxes=tf.reshape(bbox, (tf.shape(bbox)[0], -1, 1, 4)),
#         scores=tf.reshape(
#             scores, (tf.shape(scores)[0], -1, tf.shape(scores)[-1])),
#         max_output_size_per_class=100,
#         max_total_size=100,
#         iou_threshold=yolo_iou_threshold,
#         score_threshold=yolo_score_threshold)
#     return boxes, scores, classes, valid_detections

In [8]:
# def YoloV3(size=None, chan=3, anchors=y_anch,masks=y_anchor_mask, classes=80, training=False):
#     x = inputs = Input([size, size, chan])
#     x_36, x_61, x = Darknet(name='yolo_darknet')(x)
#     x = YoloConv(512, name='yolo_conv_0')(x)
#     output_0 = YoloOutput(512, len(masks[0]), classes, name='yolo_output_0')(x)
#     x = YoloConv(256, name='yolo_conv_1')((x, x_61))
#     output_1 = YoloOutput(256, len(masks[1]), classes, name='yolo_output_1')(x)
#     x = YoloConv(128, name='yolo_conv_2')((x, x_36))
#     output_2 = YoloOutput(128, len(masks[2]), classes, name='yolo_output_2')(x)
#     if training:
#         return Model(inputs, (output_0, output_1, output_2), name='yolov3')
#     return Model(inputs, outputs, name='yolov3')

In [9]:
# def YoloLoss(anchors, classes=80, ignore_thresh=0.5):
#     def yolo_loss(y_true, y_pred):
#         pred_box, pred_obj, pred_class, pred_xywh = yolo_boxes(
#             y_pred, anchors, classes)
#         pred_xy = pred_xywh[..., 0:2]
#         pred_wh = pred_xywh[..., 2:4]
#         t_box, true_obj, true_class_idx = tf.split(
#             y_true, (4, 1, 1), axis=-1)
#         tr_xy = (t_box[..., 0:2] + t_box[..., 2:4]) / 2
#         t_wh = t_box[..., 2:4] - t_box[..., 0:2]
#         b_loss = 2 - t_wh[..., 0] * t_wh[..., 1]
#         g_size = tf.shape(y_true)[1]
#         grid = tf.meshgrid(tf.range(g_size), tf.range(g_size))
#         grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
#         tr_xy = tr_xy * tf.cast(g_size, tf.float32) - tf.cast(grid, tf.float32)
#         t_wh = tf.math.log(t_wh / anchors)
#         t_wh = tf.where(tf.math.is_inf(t_wh),
#                       tf.zeros_like(t_wh), t_wh)
#         obj_mask = tf.squeeze(true_obj, -1)
#         t_box_flat = tf.boolean_mask(t_box, tf.cast(obj_mask, tf.bool))
#         best_iou = tf.reduce_max(intersectionOverUnion(
#             pred_box, true_box_flat), axis=-1)
#         ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
#         xy_loss = obj_mask * b_loss * tf.reduce_sum(tf.square(true_xy - pred_xy), axis=-1)
#         wh_loss = obj_mask * b_loss * tf.reduce_sum(tf.square(true_wh - pred_wh), axis=-1)
#         o_loss = binary_crossentropy(true_obj, pred_obj)
#         o_loss = obj_mask * o_loss + (1 - obj_mask) * ignore_mask * o_loss
#         class_loss = obj_mask * sparse_categorical_crossentropy(
#             true_class_idx, pred_class)
#         xy_loss = tf.reduce_sum(xy_loss, axis=(1, 2, 3))
#         wh_loss = tf.reduce_sum(wh_loss, axis=(1, 2, 3))
#         o_loss = tf.reduce_sum(o_loss, axis=(1, 2, 3))
#         class_loss = tf.reduce_sum(class_loss, axis=(1, 2, 3))
#         return xy_loss + wh_loss + o_loss + class_loss
#     return yolo_loss

In [10]:
# def transform_targets_for_output(y_true, grid_size, anchor_idxs, classes):
#     N = tf.shape(y_true)[0]
#     y_true_out = tf.zeros(
#       (N, grid_size, grid_size, tf.shape(anchor_idxs)[0], 6))
#     anchor_idxs = tf.cast(anchor_idxs, tf.int32)
#     indexes = tf.TensorArray(tf.int32, 1, dynamic_size=True)
#     updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)
#     idx = 0
#     for i in tf.range(N):
#         for j in tf.range(tf.shape(y_true)[1]):
#             if tf.equal(y_true[i][j][2], 0):
#                 continue
#             anchor_eq = tf.equal(
#                 anchor_idxs, tf.cast(y_true[i][j][5], tf.int32))
#             if tf.reduce_any(anchor_eq):
#                 box = y_true[i][j][0:4]
#                 anchor_idx = tf.cast(tf.where(anchor_eq), tf.int32)
#                 grid_xy = tf.cast(b_x_y // (1/grid_size), tf.int32)
#                 indexes = indexes.write(
#                     idxes, [i, grid_xy[1], grid_xy[0], anchor_idx[0][0]])
#                 idxes += 1
#             return tf.tensor_scatter_nd_update(
#         y_true_out, indexes.stack(), updates.stack())
# def transform_targets(y_train, anchors, anchor_masks, classes):
#     outputs = []
#     grid_size = 13
#     anchors = tf.cast(anchors, tf.float32)
#     a_area = anchors[..., 0] * anchors[..., 1]
#     b_wh = y_train[..., 2:4] - y_train[..., 0:2]
#     b_wh = tf.tile(tf.expand_dims(box_wh, -2),
#                     (1, 1, tf.shape(anchors)[0], 1))
#     b_area = b_wh[..., 0] * box_wh[..., 1]
#     inters = tf.minimum(b_wh[..., 0], anchors[..., 0]) * tf.minimum(b_wh[..., 1], anchors[..., 1])
#     iou = inters / (box_area + a_area - inters)
#     anchor_idx = tf.cast(tf.argmax(iou, axis=-1), tf.float32)
#     anchor_idx = tf.expand_dims(anchor_idx, axis=-1)
#     y_train = tf.concat([y_train, anchor_idx], axis=-1)
#     for anchor_idxs in anchor_masks:
#         outputs.append(transform_targets_for_output(
#             y_train, grid_size, anchor_idxs, classes))
#         grid_size *= 2
#         return tuple(outputs) 
    
# def preprocess_image(x_train, size):
#     return (tf.image.resize(x_train, (size, size))) / 255

In [11]:
# yolo = YoloV3(classes=num_classes)
# load_darknet_weights(yolo, weightyolov3)
# yolo.save_weights(checkpoints)
# def detect_objects(img_path, white_list=None):
#     image = img_path     
#     img = tf.image.decode_image(open(image, 'rb').read(), channels=3)
#     img = tf.expand_dims(img, 0)
#     img = preprocess_image(img, size)
#     bx, scor, label, num = yolo(img)
#     img = cv2.imread(image)
#     img = draw_outputs(img, (bx, scor, label, num), class_names, white_list)
#     cv2.imwrite('detected_{:}'.format(img_path), img)
#     detected = Image.open('detected_{:}'.format(img_path))
#     detected.show()
# detect_objects('test.jpg', ['bear'])

In [12]:
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
 
def _conv_block(inp, convs, skip=True):
	x = inp
	count = 0
	for conv in convs:
		if count == (len(convs) - 2) and skip:
			skip_connection = x
		count += 1
		if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
		x = Conv2D(conv['filter'],
				   conv['kernel'],
				   strides=conv['stride'],
				   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
				   name='conv_' + str(conv['layer_idx']),
				   use_bias=False if conv['bnorm'] else True)(x)
		if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
		if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
	return add([skip_connection, x]) if skip else x
 
def make_yolov3_model():
	input_image = Input(shape=(None, None, 3))
	# Layer  0 => 4
	x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
								  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
								  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
								  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
	# Layer  5 => 8
	x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
						{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
	# Layer  9 => 11
	x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
	# Layer 12 => 15
	x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
						{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
						{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
	# Layer 16 => 36
	for i in range(7):
		x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
							{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
	skip_36 = x
	# Layer 37 => 40
	x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
	# Layer 41 => 61
	for i in range(7):
		x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
							{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
	skip_61 = x
	# Layer 62 => 65
	x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
	# Layer 66 => 74
	for i in range(3):
		x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
							{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
	# Layer 75 => 79
	x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
	# Layer 80 => 82
	yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
							  {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
	# Layer 83 => 86
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_61])
	# Layer 87 => 91
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
	# Layer 92 => 94
	yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
							  {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
	# Layer 95 => 98
	x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_36])
	# Layer 99 => 106
	yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
							   {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
	model = Model(input_image, [yolo_82, yolo_94, yolo_106])
	return model
 
class WeightReader:
	def __init__(self, weight_file):
		with open(weight_file, 'rb') as w_f:
			major,	= struct.unpack('i', w_f.read(4))
			minor,	= struct.unpack('i', w_f.read(4))
			revision, = struct.unpack('i', w_f.read(4))
			if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
				w_f.read(8)
			else:
				w_f.read(4)
			transpose = (major > 1000) or (minor > 1000)
			binary = w_f.read()
		self.offset = 0
		self.all_weights = np.frombuffer(binary, dtype='float32')
 
	def read_bytes(self, size):
		self.offset = self.offset + size
		return self.all_weights[self.offset-size:self.offset]
 
	def load_weights(self, model):
		for i in range(106):
			try:
				conv_layer = model.get_layer('conv_' + str(i))
				print("loading weights of convolution #" + str(i))
				if i not in [81, 93, 105]:
					norm_layer = model.get_layer('bnorm_' + str(i))
					size = np.prod(norm_layer.get_weights()[0].shape)
					beta  = self.read_bytes(size) # bias
					gamma = self.read_bytes(size) # scale
					mean  = self.read_bytes(size) # mean
					var   = self.read_bytes(size) # variance
					weights = norm_layer.set_weights([gamma, beta, mean, var])
				if len(conv_layer.get_weights()) > 1:
					bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel, bias])
				else:
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel])
			except ValueError:
				print("no convolution #" + str(i))
 
	def reset(self):
		self.offset = 0
 




In [13]:
model = make_yolov3_model()
# load the model weights
weight_reader = WeightReader('../weights/yolov3 .weights')
# set the model weights into the model
weight_reader.load_weights(model)
# save the model to file
model.save('../models/yoloV3.h5')


loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

C:\Users\Steve\anaconda3\envs\Birds\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [17]:
from keras.models import load_model
import numpy as np
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle
 
class BoundBox:
	def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax
		self.objness = objness
		self.classes = classes
		self.label = -1
		self.score = -1
 
	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)
 
		return self.label
 
	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]
 
		return self.score
 
def _sigmoid(x):
	return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
	grid_h, grid_w = netout.shape[:2]
	nb_box = 3
	netout = netout.reshape((grid_h, grid_w, nb_box, -1))
	nb_class = netout.shape[-1] - 5
	boxes = []
	netout[..., :2]  = _sigmoid(netout[..., :2])
	netout[..., 4:]  = _sigmoid(netout[..., 4:])
	netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
	netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
	for i in range(grid_h*grid_w):
		row = i / grid_w
		col = i % grid_w
		for b in range(nb_box):
			# 4th element is objectness score
			objectness = netout[int(row)][int(col)][b][4]
			if(objectness.all() <= obj_thresh): continue
			# first 4 elements are x, y, w, and h
			x, y, w, h = netout[int(row)][int(col)][b][:4]
			x = (col + x) / grid_w # center position, unit: image width
			y = (row + y) / grid_h # center position, unit: image height
			w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
			h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
			# last elements are class probabilities
			classes = netout[int(row)][col][b][5:]
			box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
			boxes.append(box)
	return boxes
 
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
	new_w, new_h = net_w, net_h
	for i in range(len(boxes)):
		x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
		y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
		boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
		boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
		boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
		boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
 
def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3
 
def bbox_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union
 
def do_nms(boxes, nms_thresh):
	if len(boxes) > 0:
		nb_class = len(boxes[0].classes)
	else:
		return
	for c in range(nb_class):
		sorted_indices = np.argsort([-box.classes[c] for box in boxes])
		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]
			if boxes[index_i].classes[c] == 0: continue
			for j in range(i+1, len(sorted_indices)):
				index_j = sorted_indices[j]
				if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
					boxes[index_j].classes[c] = 0
 
# load and prepare an image
def load_image_pixels(filename, shape):
	# load the image to get its shape
	image = load_img(filename)
	width, height = image.size
	# load the image with the required size
	image = load_img(filename, target_size=shape)
	# convert to numpy array
	image = img_to_array(image)
	# scale pixel values to [0, 1]
	image = image.astype('float32')
	image /= 255.0
	# add a dimension so that we have one sample
	image = expand_dims(image, 0)
	return image, width, height
 
# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
	v_boxes, v_labels, v_scores = list(), list(), list()
	# enumerate all boxes
	for box in boxes:
		# enumerate all possible labels
		for i in range(len(labels)):
			# check if the threshold for this label is high enough
			if box.classes[i] > thresh:
				v_boxes.append(box)
				v_labels.append(labels[i])
				v_scores.append(box.classes[i]*100)
				# don't break, many labels may trigger for one box
	return v_boxes, v_labels, v_scores
 
# draw all results
def draw_boxes(filename, v_boxes, v_labels, v_scores):
	# load the image
	data = pyplot.imread(filename)
	# plot the image
	pyplot.imshow(data)
	# get the context for drawing boxes
	ax = pyplot.gca()
	# plot each box
	for i in range(len(v_boxes)):
		box = v_boxes[i]
		# get coordinates
		y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
		# calculate width and height of the box
		width, height = x2 - x1, y2 - y1
		# create the shape
		rect = Rectangle((x1, y1), width, height, fill=False, color='red')
		# draw the box
		ax.add_patch(rect)
		# draw text and score in top left corner
		label = "%s (%.3f)" % (v_labels[i], v_scores[i])
		pyplot.text(x1, y1, label, color='red')
	# show the plot
	pyplot.show()
 
# load yolov3 model
model = load_model('../models/yoloV3.h5')
# define the expected input shape for the model
input_w, input_h = 416, 416
# define our new photo
photo_filename = '../data/Appelvink_Man/Appelvink_Man_0.mp4'
# load and prepare image
image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
# make prediction
yhat = model.predict(image)
# summarize the shape of the list of arrays
print([a.shape for a in yhat])
# define the anchors
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
# define the probability threshold for detected objects
class_threshold = 0.6
boxes = list()
for i in range(len(yhat)):
	# decode the output of the network
	boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
# correct the sizes of the bounding boxes for the shape of the image
correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
# suppress non-maximal boxes
do_nms(boxes, 0.5)
# define the labels
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
	"boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
	"bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
	"backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
	"sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
	"tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
	"apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
	"chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
	"remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
	"book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]
# get the details of the detected objects
v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
# summarize what we found
for i in range(len(v_boxes)):
	print(v_labels[i], v_scores[i])
# draw what we found
draw_boxes(photo_filename, v_boxes, v_labels, v_scores)



UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000184E7E2EDB0>